# FAQ para Suporte Interno FAISS, LangChain e Ollama (llama3.2)
* **Descrição:** Criar um script base em Python, capaz de ler o arquivo com perguntas e respostas sobre a empresa, indexar as informações no FAISS DB, fazer Q&A com o ollama, usando LangChain para orquestrar tudo
* **Dataset:** company-info.json incluído no projeto

## Passo a passo
1. Puxar um modelo do llama usando ollama (ex: llama3.2)
2. Servir localmente o modelo usando
```bash
    ollama serve
```
3. Carregar as perguntas e respostas do JSON
4. Indexar no FAISS a resposta como conteúdo e a pergunta 
5. Criar as peças para o chain (embeding, retriever)
6. Montar o chain
7. Executar pesquisas

In [ ]:
# Importar dependências
import pandas as pd
import json
from langchain_community.llms import Ollama          # LLM local
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# 1. Carregar os dados do json
with open('company-info.json', 'r') as file:
    data = json.load(file)
    
# Converter os dados em Documents para indexação
documents = []
for item in data:
    # Usamos a resposta como conteúdo e a pergunta como metadados
    doc = Document(
        page_content=item['answer'],
        metadata={'question': item['question']}
    )
    documents.append(doc)

print(f"Carregados {len(documents)} documentos")

In [ ]:
# 2. Criar OllamaEmbeddings com o modelo que vai usar (vai usar localmente)
embeddings = OllamaEmbeddings(model="llama3.2")
print("Embeddings configurado com modelo llama3.2")

In [ ]:
# 3. Indexar dados com FAISS
vectorstore = FAISS.from_documents(documents, embeddings)
print("Documentos indexados com sucesso no FAISS")

In [ ]:
# 4. Definir retriever que vai puxar os dados
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # Retorna os 3 documentos mais relevantes
)
print("Retriever configurado")

In [ ]:
# 5. Configurar o LLM e Chain
# Configurar o modelo Ollama local
llm = Ollama(model="llama3.2")

# Definir o template de prompt para o RAG
prompt_template = """
Use the following context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Answer:
"""

PROMPT = PromptTemplate(
    template=prompt_template, 
    input_variables=["context", "question"]
)

# Criar o chain de RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

print("LLM e Chain configurados com sucesso")

In [ ]:
def ask(question):
    # 6. Desenvolver a função de Q&A que usa o chain
    print(f"\nPergunta: {question}\n")
    
    # Executa a consulta no chain
    result = qa_chain({"query": question})
    
    # Mostrar a resposta
    print("Resposta:")
    print(result["result"])
    
    # Mostrar os documentos que foram usados para responder a pergunta
    print("\nDocumentos usados para a resposta:")
    for i, doc in enumerate(result["source_documents"]):
        print(f"\nDocumento {i+1}:")
        print(f"Conteúdo: {doc.page_content}")
        print(f"Pergunta original: {doc.metadata['question']}")
        
    return result

In [ ]:
# Teste
ask("What services does Capgemini offer?")